In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [3]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/train.csv")

In [4]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Data preprocessing

In [5]:
# to do the preprocessing I create a copy of the datasets
train_copy = train_data.copy()
test_copy = test_data.copy()

# missing values
train_copy = train_copy.fillna('')
test_copy = test_copy.fillna('')

In [6]:
#Choosing vocabulary size to be 5000
voc_size = 5000

# create a new column with title and author
train_copy['final'] = train_copy['author'] + ' ' + train_copy['title']
test_copy['final'] = test_copy['author'] + ' ' + test_copy['title']

In [7]:
# stopwords
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\domy-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
#Stemming map words to their root forms
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [9]:
#Applying stemming and some preprocessing
for i in range(len(train_copy)):
  review = re.sub('[^a-zA-Z]',' ',train_copy['final'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [10]:
#Applying stemming and some preprocessing for test data
corpus_test = []
for i in range(len(test_copy)):
  review = re.sub('[^a-zA-Z]',' ',test_copy['final'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [11]:
# Converting to one hot representation
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [14]:
#Padding Sentences to make them of same size
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

In [17]:
X = train_copy.drop('label',axis=1)
y=train_copy['label']
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


# Creating and training model

In [13]:
# I have used embedding layers with LSTM
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                6464      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [18]:
#Converting into numpy array
X_final = np.array(embedded_docs)
y_final = np.array(y)
test_final = np.array(embedded_docs_test)
X_final.shape,y_final.shape,test_final.shape

((20800, 25), (20800,), (20800, 25))

In [19]:
#training model
model.fit(X_final,y_final,epochs=20,batch_size=64)

Epoch 1/20
325/325 [==============================] - 8s 18ms/step - loss: 0.3367 - accuracy: 0.8423
Epoch 2/20
325/325 [==============================] - 6s 19ms/step - loss: 0.0277 - accuracy: 0.9918
Epoch 3/20
325/325 [==============================] - 6s 18ms/step - loss: 0.0106 - accuracy: 0.9974
Epoch 4/20
325/325 [==============================] - 6s 18ms/step - loss: 0.0063 - accuracy: 0.9980
Epoch 5/20
325/325 [==============================] - 6s 18ms/step - loss: 0.0037 - accuracy: 0.9990
Epoch 6/20
325/325 [==============================] - 6s 18ms/step - loss: 0.0024 - accuracy: 0.9993
Epoch 7/20
325/325 [==============================] - 5s 17ms/step - loss: 0.0037 - accuracy: 0.9989
Epoch 8/20
325/325 [==============================] - 6s 17ms/step - loss: 0.0020 - accuracy: 0.9996
Epoch 9/20
325/325 [==============================] - 6s 17ms/step - loss: 0.0014 - accuracy: 0.9994
Epoch 10/20
325/325 [==============================] - 6s 17ms/step - loss: 0.0015 - accura

In [32]:
#results = model.predict_classes(test_final)
y_pred = (model.predict(test_final) > 0.5).astype("int32")

final_sub = pd.DataFrame()
final_sub['id']=test_copy['id']
final_sub['label'] = y_pred
print(final_sub.head())

   id  label
0   0      1
1   1      0
2   2      1
3   3      1
4   4      1
